In [119]:
import os, sys, glob
import numpy as np
import scipy.signal as sigpro
import scipy.linalg as scpla

from tqdm.notebook import tqdm

%matplotlib notebook
import matplotlib.pyplot as plt

# Problem 7